In [51]:
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display

import glob
import numpy as np
import matplotlib.pyplot as plt

import invisible_cities.io.dst_io as dstio

import invisible_cities.core.core_functions as coref
import invisible_cities.core.fit_functions as fitf

from invisible_cities.icaro.hst_functions import hist
from invisible_cities.core.core_functions import shift_to_bin_centers
from invisible_cities.core.core_functions import weighted_mean_and_std

In [3]:
%matplotlib inline

np.warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"]          = 8, 6
plt.rcParams["font.size"]               = 15
plt.rcParams["figure.max_open_warning"] = 100

profOpt = "--k"
fitOpt  = "r"

In [104]:
def energySpectrum_RZcut(kdst, rlim, zlim, xlim, ylim, tlim):
    fig, ax = plt.subplots(2, 3, figsize=(24,12))
    sel = (coref.in_range(kdst.R, rlim[0], rlim[1])) & (coref.in_range(kdst.Z, zlim[0], zlim[1]))
    sel = sel & (coref.in_range(kdst.X, xlim[0], xlim[1])) & (coref.in_range(kdst.Y, ylim[0], ylim[1]))
    sel = sel & (coref.in_range(kdst.time, tlim[0], tlim[1]))
    
    #S1e plot
    y, x, _ = ax[0,0].hist(kdst.S1e[sel], 100, [0000, 1.5*kdst.S1e[sel].median()] , histtype  = 'step', linewidth = 1.5, color='k')
    x=shift_to_bin_centers(x)
    ax[0,0].set_xlabel('S1 energy (pes)')
    ax[0,0].set_ylabel('Entries')
    
    mean, std = weighted_mean_and_std(x, y, frequentist = True, unbiased = True)
    entries_string = f'Entries = {np.sum(y):.0f}\n'
    mean_string    = f'Mean = {mean:.2f}\n'
    rms_string     = f'RMS  = {std:.2f}'
    
    ax[0,0].annotate(entries_string + mean_string + rms_string,
                     xy                  = (0.99, 0.99),
                     xycoords            = 'axes fraction',
                     fontsize            = 11,
                     weight              = 'bold',
                     color               = 'black',
                     horizontalalignment = 'right',
                     verticalalignment   = 'top')    
    ax[0,0].grid     (True)
    
    # S2e plot
    emax = 1.5*kdst.S2e[sel].median()
    y, x, _ = ax[0,1].hist(kdst.S2e[sel], 100, [0000, emax], histtype  = 'step', linewidth = 1.5, color='k')
    x=shift_to_bin_centers(x)
    ax[0,1].set_xlabel('S2 energy (pes)')
    ax[0,1].set_ylabel('Entries')
    
    mean, std = weighted_mean_and_std(x, y, frequentist = True, unbiased = True)
    entries_string = f'Entries = {np.sum(y):.0f}\n'
    mean_string    = f'Mean = {mean:.2f}\n'
    rms_string     = f'RMS  = {std:.2f}'
    
    ax[0,1].annotate(entries_string + mean_string + rms_string,
                     xy                  = (0.99, 0.99),
                     xycoords            = 'axes fraction',
                     fontsize            = 11,
                     weight              = 'bold',
                     color               = 'black',
                     horizontalalignment = 'right',
                     verticalalignment   = 'top')
    ax[0,1].grid     (True)
 
    #seed = np.max(y), x[np.argmax(y)], np.std(y)
    #f    = fitf.fit(fitf.gauss, x, y, seed, fit_range=[x[np.argmax(y)]-1000,x[np.argmax(y)]+1000])
    #ax[0,0].plot(x, f.fn(x), 'r')
    #ax[0,0].annotate(f'R = {100*f.values[2]*2.35/f.values[1]:.2f}%', xy=(0.01, 0.99), xycoords='axes fraction', fontsize=11, weight='bold')

    #S2q plot
    y, x, _ = ax[0,2].hist(kdst.S1e[sel], 100, [0000, 1.5*kdst.S2q[sel].median()] , histtype  = 'step', linewidth = 1.5, color='k')
    x=shift_to_bin_centers(x)
    ax[0,2].set_xlabel('S2 charge (pes)')
    ax[0,2].set_ylabel('Entries')
    
    mean, std = weighted_mean_and_std(x, y, frequentist = True, unbiased = True)
    mean_string    = f'Mean = {mean:.2f}\n'
    rms_string     = f'RMS  = {std:.2f}'
    
    ax[0,2].annotate(mean_string + rms_string,
                     xy                  = (0.99, 0.99),
                     xycoords            = 'axes fraction',
                     fontsize            = 11,
                     weight              = 'bold',
                     color               = 'black',
                     horizontalalignment = 'right',
                     verticalalignment   = 'top')    
    ax[0,2].grid     (True)

    #E vs R plot
    ax[1,0].hist2d(kdst.R[sel], kdst.S2e[sel], 100, [[0, 200],[0000, emax]])
    ax[1,0].set_xlabel('R (mm)')
    ax[1,0].set_ylabel('S2 energy (pes)')
    
    #E vs Z plot
    ax[1,1].hist2d(kdst.Z[sel], kdst.S2e[sel], 100, [[0, 600],[0000, emax]])
    ax[1,1].set_xlabel('Drift time (µs)')
    ax[1,1].set_ylabel('S2 energy (pes)')
    
    #E vs time plot
    ax[1,2].hist2d(kdst.time[sel], kdst.S2e[sel], 100, [[0, kdst.time.max()],[0000, emax]])
    ax[1,2].set_xlabel('Running time (min)')
    ax[1,2].set_ylabel('S2 energy (pes)')    
    
    
    fig.tight_layout()

In [99]:
def defineRangeSlider(min_val, max_val, title, step=0.1):
    layout = {'width':'20%', 'height': '20px'}
    return widgets.FloatRangeSlider(value=[min_val, max_val],
                                    min=min_val,
                                    max=max_val,
                                    step=step,
                                    description=title,
                                    disabled=False,
                                    continuous_update=False,
                                    orientation='horizontal',
                                    readout=True,
                                    readout_format='.1f',
                                    layout=layout)

In [100]:
inpath = glob.glob('/home/lior/NEXT/Software/data/Run4/6181/kdst/kdst*')

kdst = dstio.load_dsts(inpath, 'DST', 'Events')

In [101]:
kdst.time = (kdst.time - kdst.time.min())/60

In [106]:
z = defineRangeSlider(0, 600, 'Z (µs)', 10)
r = defineRangeSlider(0, 200, 'R (mm)', 10)
x = defineRangeSlider(-200, 200, 'X (mm)', 10)
y = defineRangeSlider(-200, 200, 'Y (mm)', 10)
t = defineRangeSlider(0, kdst.time.max(), 'Time (min)', 30)

In [107]:
w = interactive(energySpectrum_RZcut, kdst=fixed(kdst), rlim=r, zlim=z, xlim=x, ylim=y, tlim=t)
display(w)

interactive(children=(FloatRangeSlider(value=(0.0, 200.0), continuous_update=False, description='R (mm)', layo…

`w` is a widget object that contains the current values results, example for `rmax`:

In [31]:
w.kwargs['rmax']

140